In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
import os.path as op
#from ccfj import GetStationPairs
from geopy.distance import great_circle
#import shapefile
import geopandas as gp
import yaml
import math
import shutil

In [24]:
from BayHunter import PlotFromStorage
from BayHunter import Targets
from BayHunter import utils
from BayHunter import MCMC_Optimizer
from BayHunter import ModelMatrix
from BayHunter import SynthObs
import logging,sys,getopt

In [25]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import mathlib

In [26]:
# console printout formatting
formatter = ' %(processName)-12s: %(levelname)-8s |  %(message)s'
logging.basicConfig(format=formatter, level=logging.INFO)
logger = logging.getLogger()
initial_model = 'initial_model_1.txt'

In [27]:
flag_project = 1 # 0--regular; 1--repartrition

In [28]:
if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']

In [29]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/


In [30]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [31]:
dir_partition = dir_project + info_basic['rdir_partition']
dir_data = dir_project + info_basic['rdir_disp_pick']
dir_inv_MCMC = dir_project + info_basic['rdir_inv_McMC']
outname_config = dir_inv_MCMC + 'config_McMC.ini'

In [32]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [33]:
nums = [1]
key_subworks_all = info_basic['key_subworks']
key_subworks = []
for num in nums:
    for key_subwork in key_subworks_all:
        if str(num) == key_subwork.split('--')[0]:
            key_subworks.append(key_subwork)
            break
#key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

['1--08-03']

In [34]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = mathlib.GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 
elevation_stations_all = stainfo['elevation'].tolist()

In [35]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
num_stations = []
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    num_stations.append(len(stations_this))
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

In [36]:
key_subwork = key_subworks[0]

In [37]:
modes_number = 1

In [38]:
# create init
priors, initparams = utils.load_params(outname_config)
refinitmodel = {}
refdep, refvs = np.loadtxt(dir_inv_MCMC+'/initial/'+initial_model, usecols=[1, 3]).T # should conside the half space 
refinitmodel = {'model': (refdep, refvs),'nlays': len(refdep)}
prefdep = np.concatenate((np.repeat(refdep, 2)[1:], [0.3]))
prefvs = np.repeat(refvs, 2)
initmodel = {'model': (prefdep, prefvs),'nlays': len(prefdep)-1}

In [39]:
# load disp data
file_data = dir_data + 'ds_'+key_subwork+'curve.txt'
disp_data = np.loadtxt(file_data, dtype='float', unpack=True).T
xsws=dict();ysws=dict()
modes = list(set(disp_data[:,2].astype(int)))
for m in modes:
    d = disp_data[disp_data[:,2] == m]
    freqs = d[:,0]; vels = d[:,1]
    xsws[m]=1.0/freqs[::-1]; ysws[m]=vels[::-1]

In [40]:
# Definee the target
jointargets = []
if modes_number > modes[-1]:
    modes_used = modes
else:
    modes_used = [num for num in range(modes_number+1)]

for m in modes_used:
    target_swd = Targets.RayleighDispersionPhase(xsws[m], ysws[m])
    #target_swd = Targets.LoveDispersionPhase(xsws[m], ysws[m])
    target_swd.moddata.plugin.set_modelparams(mode=m+1)
    print("mode: ", target_swd.moddata.plugin.modelparams['mode'])
    jointargets.append(target_swd)
targets = Targets.JointTarget(jointargets)

 MainProcess : INFO     |  Initiated target: RayleighDispersionPhase (ref: rdispph)


Your observed data vector exceeds the maximum of 60 periods that is allowed in SurfDisp. For forward modeling SurfDisp will reduce the samples to 60 by linear interpolation within the given period span.
From this data, the dispersion velocities to your observed periods will be determined. The precision of the data will depend on the distribution of your samples and the complexity of the input velocity-depth model.
mode:  1


In [41]:
# parameter update
savepath = dir_inv_MCMC + 'inversion/' + key_subwork + '/'
if not os.path.exists(savepath):
    os.makedirs(savepath)
priors.update({
               #'mohoest': (38, 4),  # optional, moho estimate (mean, std)
               'rfnoise_corr': 0.98,
               'swdnoise_corr': 0.
               # 'rfnoise_sigma': np.std(yrf_err),  # fixed to true value
               # 'swdnoise_sigma': np.std(ysw_err),  # fixed to true value
               })
initparams.update({'nchains': 10,
                   'iter_burnin': (2048 * 32),
                   'iter_main': (2048 * 16),
                   'propdist': (0.030, 0.0025, 0.015, 0.005), # vs, depth, layer brith or death, noise lavel
                   'savepath': savepath
                   })

In [42]:
# MCMC BAY INVERSION
optimizer = MCMC_Optimizer(targets, initparams=initparams, priors=priors,
                           refinitmodel = refinitmodel, random_seed=None)
optimizer.mp_inversion(nthreads=10, baywatch=False, dtsend=1)

 MainProcess : INFO     |  > Chain arrays are initiated...
 MainProcess : INFO     |  ... they occupy ~363 MB memory.
 MainProcess : INFO     |  > 10 chain(s) are initiated ...
 MainProcess : INFO     |  > Sending out chain 0
 MainProcess : INFO     |  > Sending out chain 1
 MainProcess : INFO     |  > Sending out chain 2
 MainProcess : INFO     |  > Sending out chain 3
 MainProcess : INFO     |  > Sending out chain 4
 MainProcess : INFO     |  > Sending out chain 5
 MainProcess : INFO     |  > Sending out chain 6
 MainProcess : INFO     |  > Sending out chain 7
 MainProcess : INFO     |  > Sending out chain 8
 MainProcess : INFO     |  > Sending out chain 9
 chain 8     : INFO     |  -65001    99 + hs    0.064	      217 |  10.6 s  | 53.9 %
 chain 0     : INFO     |  -65000    99 + hs    0.057	      250 |  10.9 s  | 58.2 %
 chain 4     : INFO     |  -65000    99 + hs    0.054	      266 |  10.8 s  | 55.8 %
 chain 5     : INFO     |  -65000    99 + hs    0.043	      310 |  10.9 s  | 56.3

In [45]:
# Model resaving and plotting
path = initparams['savepath']
cfile = '%s_config.pkl' % initparams['station']
configfile = op.join(path, 'data', cfile)
obj = PlotFromStorage(configfile)
# The final distributions will be saved with save_final_distribution.
# Beforehand, outlier chains will be detected and excluded.
# Outlier chains are defined as chains with a likelihood deviation
# of dev * 100 % from the median posterior likelihood of the best chain.
obj.save_final_distribution(maxmodels=100000, dev=1)
obj.save_posterior_models()

Current data path: ./project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/inverion_McMC/inversion/1--08-03/data
Outlier chains from file: None
> Saving posterior distribution.
./project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/inverion_McMC/inversion/1--08-03/data/c_models
./project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/inverion_McMC/inversion/1--08-03/data/c_likes
./project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/inverion_McMC/inversion/1--08-03/data/c_misfits
./project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/inverion_McMC/inversion/1--08-03/data/c_noise


In [46]:
#--------------------------plot multi-modes dispersion curves !!!!!!!!!!!!!!!!!!!!!!
modes_plot = []
modes_plot = [x for x in modes if x not in modes_used]
jointargets_plot = []
if modes_plot:
    for m in modes_plot:
        target_swd = Targets.LoveDispersionPhase(xsws[m], ysws[m])
        target_swd.moddata.plugin.set_modelparams(mode=m+1)
        print("mode: ", target_swd.moddata.plugin.modelparams['mode'])
        jointargets_plot.append(target_swd)

if priors['refinit'] is not None: 
    obj.save_plots(initmodel=initmodel, jointargets_plot=jointargets_plot, nchains=10)
else:
    obj.save_plots(jointargets_plot=jointargets_plot, nchains=10)

obj.merge_pdfs()

 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family 'Arial' not found.
 MainProcess : WARNING  |  findfont: Font family